In [2]:
import pandas as pd
import numpy as np

# Clean and Combine Data
##### The following is cleaning and preparing data. It is already done. Please do not run it again.

In [3]:
# Load and clean data from Steam Charts data file, scrapped by Alex. Do not run this again.
data = pd.read_csv('GamesData.csv',index_col=False)

# Clean data
data = data.iloc[2:].reset_index(drop=True)#.drop(columns='Unnamed: 0')

# Make percentile into number and clear "-"
data = data.replace(to_replace='%', value='', regex=True).replace({'-':'NaN'})
data = data[2:].reset_index(drop=True).drop(columns='Unnamed: 0')

data = data.rename(columns=lambda x: x.replace('_', ' '))
data = data.rename(columns={'App id':'App_id','Avg Players':'Avg_Players', 'Peak Players':'Peak_Players','Perc Gain': 'Perc_Gain'}, errors="raise")

# Save the cleaned data as 'SteamCharts.csv'
data.to_csv('SteamCharts.csv',index=False)

In [3]:
# Load data from 'SteamCharts.csv' data file
data = pd.read_csv('SteamCharts.csv',index_col=False,low_memory=False)

# Get Perc_Gain.
p_gain = data.loc[data['Perc_Gain']==1]
p_gain = p_gain.drop(columns=['Avg_Players', 'Gain', 'Perc_Gain', 'Peak_Players','Last 30 Days'])
p_gain_pure = p_gain.drop(columns=['App_id', 'Name']).astype(float)

# Get Avg_Players.
avg_players = data.loc[data['Avg_Players']==1]
avg_players = avg_players.drop(columns=['Avg_Players', 'Gain', 'Perc_Gain', 'Peak_Players','Last 30 Days'])
avg_pure = avg_players.drop(columns=['App_id', 'Name']).astype(float)

# Get Peak_Players.
peak_players = data.loc[data['Peak_Players']==1]
peak_players = peak_players.drop(columns=['Avg_Players', 'Gain', 'Perc_Gain', 'Peak_Players','Last 30 Days'])
peak_pure = peak_players.drop(columns=['App_id', 'Name']).astype(float)

# Add columns of max and mean
avg_players['max'] = avg_pure.max(axis=1)
avg_players['mean'] = avg_pure.mean(axis=1)

peak_players['peak_max'] = peak_pure.max(axis=1)
peak_players['peak_mean'] = peak_pure.mean(axis=1)

In [4]:
# Load data from Steam Spy data files. Do not run this, or download 'steamspy_owners.csv' and 'games-features-metacritic-new.csv' before you run:
steamspy = pd.read_csv('steamspy_owners.csv')
steamspy = steamspy.rename(columns={'QueryID':'App_id','0':'SteamSpyOwnersNew'}, errors="raise")

features = pd.read_csv('games-features-metacritic-new.csv',index_col=False)
features = features.drop_duplicates(subset=['QueryID','QueryName'], keep='first', inplace=False, ignore_index=False)
features = features.drop(columns=['ResponseID', 'ResponseName'])
features = features.rename(columns={'QueryID':'App_id','QueryName':'Name'}, errors="raise")

# Combine data from Steam Charts and Steam Spy, order it by 'max' and save it as 'AllData.csv'
data_all = pd.merge(features, steamspy[['App_id','SteamSpyOwnersNew']], how='inner', on=['App_id'])
data_all = pd.merge(data_all, peak_players[['App_id','peak_max','peak_mean']], how='inner', on=['App_id'])
data_all = pd.merge(data_all, avg_players[['App_id','max','mean']], how='inner', on=['App_id'])
data_all.sort_values(by=['max'], ascending=False, inplace=True)

data_all.reset_index(drop=True).drop(columns='Unnamed: 0').drop_duplicates(subset=['Name'], keep='first', inplace=False, ignore_index=False).to_csv('AllData.csv',index=False)

# Explanation of the data

The file `SteamCharts.csv` contains monthly data during July 2012 to April 2021 including
- 'Avg_Players': the daily average number of players per month; 
- 'Peak_Players': the daily peak number of players per month;
- 'Gain'
- 'Perc_Gain'  

obtained from [Steam Charts](https://steamcharts.com/).

The file `AllData.csv` contains 70+ game features (obtained from [Steam Spy](https://steamspy.com/)), together with the max and mean of 'Avg_Players' and the max and mean of 'Peak_Players'. This file is a combination of Steam Spy data and Steam Charts data through the common 'App_id's. It is currently sorted by the max of 'Avg_Players'.
